In [1]:
import pandas as pd
import numpy as np

In [25]:
movies=pd.read_csv("movies.csv",sep=';',encoding='latin-1').drop('Unnamed: 3',axis=1)
users=pd.read_csv("users.csv",sep=';')
rating=pd.read_csv("ratings.csv",sep=';')

In [26]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
movies.shape

(3883, 3)

In [11]:
users.head()

,userId,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [20]:
users.rename(columns={'userId':'Id'},inplace=True)

In [21]:
users.head()

,Id,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [15]:
users.shape

(6040, 5)

In [27]:
rating.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [28]:
rating.shape

(1000209, 4)

In [29]:
df=movies.merge(rating, on="movieId")

In [30]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [32]:
df.shape

(1000209, 6)

In [33]:
df_movies_cnt = pd.DataFrame(df.groupby('movieId').size(), columns=['count'])
df_movies_cnt.head()

,count
movieId,
1,2077
2,701
3,478
4,170
5,296


In [34]:
df.isnull().sum()

movieId      0
title        0
genres       0
userId       0
rating       0
timestamp    0
dtype: int64

In [35]:
df = df.drop_duplicates()

In [37]:
df.shape

(1000209, 6)

In [38]:
df.iloc[0]['genres']

"Animation|Children's|Comedy"

In [68]:
print("sorted based on top 10 rating")
df.sort_values(by=["rating"], ascending=False).head()

sorted based on top 10 rating


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
657724,2409,Rocky II (1979),Action|Drama,101,5,977578472
244214,1012,Old Yeller (1957),Children's|Drama,447,5,976236279
657745,2409,Rocky II (1979),Action|Drama,549,5,976119207
657752,2409,Rocky II (1979),Action|Drama,684,5,975603281


Collaborative Filtering :
 
     It is considered to be one of the very smart recommender systems that work on the similarity between different users and also items that are widely used as an e-commerce website and also online movie websites. It checks about the taste of similar users and does recommendations.

In [43]:
# Pivot Table with respect to ratings given by users to movies..
pivot = df.pivot_table(values='rating',columns='userId',index='movieId').fillna(0)
print('Shape of this pivot table :',pivot.shape)
pivot.head()

Shape of this pivot table : (3706, 6040)


userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# Machine Learning Model training for Recommending movies based on users ratings
from sklearn.neighbors import NearestNeighbors
nn_algo = NearestNeighbors(metric='cosine')
nn_algo.fit(pivot)

NearestNeighbors(metric='cosine')

In [55]:
#Developing the class of Collaborative filtering Recommendation Engine..

class Recommender:
    def recommend_on_movie(self,movie,n_reccomend = 5):
        movieid = int(movies[movies['title']==movie]['movieId'])
        distance,neighbors = nn_algo.kneighbors([rating_pivot.loc[movieid]],n_neighbors=n_reccomend+1)
        movieids = [rating_pivot.iloc[i].name for i in neighbors[0]]
        recommeds = [str(movies[movies['movieId']==mid]['title']).split('\n')[0].split('  ')[-1] for mid in movieids if mid not in [movieid]]
        return recommeds[:n_reccomend]
    

In [56]:
recommender = Recommender()

In [57]:
recommender.recommend_on_movie('Father of the Bride Part II (1995)')

['Home Alone (1990)',
 'Home Alone 2',
 'Mighty Ducks, The (1992)',
 'Mrs. Doubtfire (1993)',
 'Liar Liar (1997)']

In [58]:
recommender.recommend_on_movie('Batman')

['Akira (1988)',
 'Transformers',
 'Ghost in the Shell (Kokaku kidotai) (1995)',
 'Aladdin (1992)',
 'Sword in the Stone, The (1963)']

In [63]:
recommender.recommend_on_movie('Old Yeller (1957)')

['Bambi (1942)',
 'Swiss Family Robinson (1960)',
 'Parent Trap, The (1961)',
 'Lady and the Tramp (1955)',
 'Dumbo (1941)']

In [64]:
recommender.recommend_on_movie('Lady and the Tramp (1955)')

['Cinderella (1950)',
 'Jungle Book, The (1967)',
 'Peter Pan (1953)',
 'Sleeping Beauty (1959)',
 'Snow White and the Seven Dwarfs (1937)']